In [1]:
import torch
import torch.nn.functional as F
import sys
import numpy as np

sys.path.append('/home/hxcai/cell_type_specific_CRE')
# from MPRA_exp.datasets import SeqLabelDataset
# from MPRA_exp.utils import *

In [18]:
# def create_embedding_with_suffix(embedding, suffix):
#     # 根据后缀创建新的列
#     new_columns = [np.ones((embedding.shape[0], 1)) * int(bit) for bit in suffix]
#     # 合并原始嵌入和新列
#     return np.concatenate([embedding] + new_columns, axis=1)

# # 创建所有后缀组合
# suffixes = ['00', '01', '10', '11']

# # 使用列表推导和函数创建所有嵌入
# embeddings_with_suffixes = [create_embedding_with_suffix(embedding, suffix) for suffix in suffixes]

# # 合并所有嵌入
# all_embeddings = np.concatenate(embeddings_with_suffixes, axis=0)

# all_embeddings.shape

In [2]:
def normalize(data):
    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    data_norm = (data - data_mean) / (data_std + 1e-6)
    return data_norm


# def del_zero_variance_dim(data):
#     variances = np.var(data, axis=0)
#     data_cleaned = data[:, variances > 0]
#     return data_cleaned

In [6]:
embedding = np.load('./data/sei_joint_embedding.npy')
print(embedding.shape)
# embedding = del_zero_variance_dim(embedding)

# 复制4份，加上onehot_encoding
onehot_encoding = np.zeros((embedding.shape[0], 4))
onehot_encoding[:] = [0,1,0,1]
HepG2_ATAC_embedding = np.concatenate([embedding, onehot_encoding], axis=1)
onehot_encoding[:] = [0,1,1,0]
K562_ATAC_embedding = np.concatenate([embedding, onehot_encoding], axis=1)
onehot_encoding[:] = [1,0,0,1]
HepG2_MPRA_embedding = np.concatenate([embedding, onehot_encoding], axis=1)
onehot_encoding[:] = [1,0,1,0]
K562_MPRA_embedding = np.concatenate([embedding, onehot_encoding], axis=1)

train_embedding = np.concatenate([HepG2_ATAC_embedding, K562_ATAC_embedding, HepG2_MPRA_embedding], axis=0)
valid_embedding = np.concatenate([K562_MPRA_embedding], axis=0)

np.save('train_embedding.npy', train_embedding)
np.save('valid_embedding.npy', valid_embedding)

(56975, 15360)


In [8]:
HepG2_ATAC_labels = np.load('data/sei_joint_pred.npy')[:, 20988]
K562_ATAC_labels = np.load('data/sei_joint_pred.npy')[:, 21326]
HepG2_MPRA_labels = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/lentiMPRA/joint_library_table_s.csv', sep=',')['HepG2']
K562_MPRA_labels = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/lentiMPRA/joint_library_table_s.csv', sep=',')['K562']

HepG2_ATAC_labels = normalize(HepG2_ATAC_labels)
K562_ATAC_labels = normalize(K562_ATAC_labels)
HepG2_MPRA_labels = normalize(HepG2_MPRA_labels)
K562_MPRA_labels = normalize(K562_MPRA_labels)

train_labels = np.concatenate([HepG2_ATAC_labels, K562_ATAC_labels, HepG2_MPRA_labels], axis=0)
valid_labels = np.concatenate([K562_MPRA_labels], axis=0)

np.save('train_labels.npy', train_labels)
np.save('valid_labels.npy', valid_labels)

In [9]:
pearsonr(HepG2_MPRA_labels, K562_MPRA_labels)

PearsonRResult(statistic=0.6142933946114568, pvalue=0.0)

In [10]:
train_embedding.shape

(170925, 15362)

In [11]:
train_labels

array([-0.83376729, -0.91402435,  1.49626064, ..., -0.89090818,
       -1.39646275, -1.13965179])